# Tree detection with DeepForest
In this notebook, we reproduce the results of tree detection step of the pipeline

In [ ]:
from typing import Tuple, Generator
import os
from math import ceil

import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from deepforest import deepforest

Define class for extracting patches from large RGB images

In [ ]:
class PatchGenerator:
    def __init__(self, image: np.ndarray, patch_shape: Tuple[int, int]) -> None:
        self.image = image
        self.height, self.width, self.num_channels = self.image.shape
        self.patch_shape = patch_shape
        self._extract_patches()

    def __call__(self) -> Generator[np.ndarray, None, None]:
        for idx in range(self._calculate_num_patches()):
            yield self.patches[idx, :, :, :]

    def get_patch(self, idx: int) -> np.ndarray:
        return self.patches[idx, :, :, :]

    def get_patch_tl_br(self, idx: int) -> Tuple[int, int, int, int]:
        i, j = self._calculate_patch_coord(idx)
        v_start, v_end = i * self.patch_shape[0], (i + 1) * self.patch_shape[0]
        u_start, u_end = j * self.patch_shape[1], (j + 1) * self.patch_shape[1]
        return u_start, v_start, u_end, v_end

    def get_num_patches(self) -> int:
        return self._calculate_num_patches()

    def stitch_patches(self, patches: np.ndarray = None) -> np.ndarray:
        if patches is None:
            patches = self.patches
        stiched_image = np.zeros_like(self.image)
        for idx in range(self._calculate_num_patches()):
            u_start, v_start, u_end, v_end = self.get_patch_tl_br(idx)
            patch_shape = stiched_image[v_start:v_end, u_start:u_end, :].shape
            stiched_image[v_start:v_end, u_start:u_end, :] = patches[idx, :patch_shape[0], :patch_shape[1], :]
        return stiched_image

    def _extract_patches(self) -> np.ndarray:
        num_patches = self._calculate_num_patches()
        self.patches = np.zeros((num_patches, *self.patch_shape, self.num_channels))

        for idx in range(num_patches):
            u_start, v_start, u_end, v_end = self.get_patch_tl_br(idx)
            patch = self.image[v_start:v_end, u_start:u_end, :]
            patch_height, patch_width, _ = patch.shape
            self.patches[idx, :patch_height, :patch_width, :] = patch

    def _calculate_num_rows_columns(self) -> Tuple[int, int]:
        num_rows = ceil(self.height / self.patch_shape[0])
        num_columns = ceil(self.width / self.patch_shape[1])
        return num_rows, num_columns

    def _calculate_patch_coord(self, idx: int) -> Tuple[int, int]:
        _, num_columns = self._calculate_num_rows_columns()
        row_idx = idx // num_columns
        col_idx = idx % num_columns
        return row_idx, col_idx

    def _calculate_num_patches(self) -> int:
        num_rows, num_columns = self._calculate_num_rows_columns()
        return num_rows * num_columns


Define method for loading images

In [ ]:
def load_image(image_filepath: str) -> np.ndarray:
    return cv2.imread(image_filepath)

## Use deepforest model to predict bounding boxes of tree crowns

In [ ]:
MODEL_FILEPATH = "./models/final_model_4000_epochs_35.h5"
PREDICTION_OUTPUT_DIR = "./data/processed/predicted_bbox"
PATCH_SHAPE = (4000, 4000)
SCORE_THRESHOLD = 0.1

Load model

In [ ]:
model = deepforest.deepforest(saved_model=MODEL_FILEPATH)

Define function to predict boudning boxes for given orthomosaic (RGB image)

In [ ]:
def predict_bounding_boxes(orthomosaic_filepath):
    orthomosaic = load_image(orthomosaic_filepath)
    patch_generator = PatchGenerator(orthomosaic, PATCH_SHAPE)
    prediction: pd.DataFrame = None
    for idx, patch in enumerate(patch_generator()):
        pred = model.predict_image(
            numpy_image=patch, return_plot=False, score_threshold=SCORE_THRESHOLD
        )
        # Transform bounding box cooords from patch to orthomosaic
        vmin, umin, _, _ = patch_generator.get_patch_tl_br(idx)
        pred['xmin'] = pred['xmin'] + vmin
        pred['xmax'] = pred['xmax'] + vmin
        pred['ymin'] = pred['ymin'] + umin
        pred['ymax'] = pred['ymax'] + umin
        prediction = pd.concat([prediction, pred], ignore_index=True)
    return prediction

Define function to save predictions

In [ ]:
def save_predictions(prediction: pd.DataFrame, orthomosaic_filepath):
    orthomosaic_filename = os.path.splitext(os.path.basename(orthomosaic_filepath))[0]
    output_filename = os.path.join(PREDICTION_OUTPUT_DIR, f"{orthomosaic_filename}_predicted_bounding_boxes.csv")
    prediction.to_csv(output_filename)

Main

In [ ]:
orthomosaic_filepaths = [
    "./data/raw/Carlos Vera Arteaga RGB.tif",
    "./data/raw/Carlos Vera Guevara RGB.tif",
    "./data/raw/Flora Pluas RGB.tif",
    "./data/raw/Leonor Aspiazu RGB.tif",
    "./data/raw/Manuel Macias RGB.tif",
    "./data/raw/Nestor Macias RGB.tif",
]

for orthomosaic_filepath in tqdm(orthomosaic_filepaths):
    predictions = predict_bounding_boxes(orthomosaic_filepath)
    print(predictions.head())
    save_predictions(predictions, orthomosaic_filepath)

Define function to generate visualizations

In [ ]:
ORTHOMOSAICS_DIR = "./data/raw/"
VISUALIZATION_OUTPUT_DIR = "./data/processed/predicted_bbox_visualizations"
COLOR = (0, 0, 255)
THICKNESS = 20

In [ ]:
def visualize_bounding_boxes(orthomosaic_name):
    # Load deepforest predictions
    prediction_filename = f"{orthomosaic_name}_predicted_bounding_boxes.csv"
    prediction_filepath = os.path.join(PREDICTION_OUTPUT_DIR, prediction_filename)
    predictions = pd.read_csv(prediction_filepath)
    # Load RGB orthomosaic
    ortho_filename = f"{orthomosaic_name}.tif"
    ortho_filepath = os.path.join(ORTHOMOSAICS_DIR, ortho_filename)
    image = load_image(ortho_filepath)
    # Draw bounding boxes
    for _, row in predictions.iterrows():
        p1 = (int(row.xmin), int(row.ymin))
        p2 = (int(row.xmax), int(row.ymax))
        image = cv2.rectangle(image, p1, p2, COLOR, THICKNESS)
    # Save visualizations
    output_filename = f"{orthomosaic_name}_predicted_boxes.png"
    output_filepath = os.path.join(VISUALIZATION_OUTPUT_DIR, output_filename)
    cv2.imwrite(output_filepath, image)
    image_compressed = cv2.resize(image, None, fx=0.1, fy=0.1)
    output_filename = f"{VISUALIZATION_OUTPUT_DIR}_predicted_boxes_compressed.png"
    output_filepath = os.path.join(VISUALIZATION_OUTPUT_DIR, output_filename)
    cv2.imwrite(output_filepath, image_compressed)

Draw and save visualizations

In [ ]:

orthomosaic_names = [
    "Carlos Vera Arteaga RGB",
    "Carlos Vera Guevara RGB",
    "Flora Pluas RGB",
    "Leonor Aspiazu RGB",
    "Manuel Macias RGB",
    "Nestor Macias RGB",
]
for ortho_name in tqdm(orthomosaic_names):
    visualize_bounding_boxes(ortho_name)